<div align="center" id="top">
<img src="https://socialify.git.ci/julep-ai/julep/image?description=1&descriptionEditable=Serverless%20AI%20Workflows%20for%20Data%20%26%20ML%20Teams&font=Source%20Code%20Pro&logo=https%3A%2F%2Fraw.githubusercontent.com%2Fjulep-ai%2Fjulep%2Fdev%2F.github%2Fjulep-logo.svg&owner=1&forks=1&pattern=Solid&stargazers=1&theme=Auto" alt="julep" />

<br>
  <p>
    <a href="https://dashboard.julep.ai">
      <img src="https://img.shields.io/badge/Get_API_Key-FF5733?style=logo=" alt="Get API Key" height="28">
    </a>
    <span>&nbsp;</span>
    <a href="https://docs.julep.ai">
      <img src="https://img.shields.io/badge/Documentation-4B32C3?style=logo=gitbook&logoColor=white" alt="Documentation" height="28">
    </a>
  </p>
  <p>
   <a href="https://www.npmjs.com/package/@julep/sdk"><img src="https://img.shields.io/npm/v/%40julep%2Fsdk?style=social&amp;logo=npm&amp;link=https%3A%2F%2Fwww.npmjs.com%2Fpackage%2F%40julep%2Fsdk" alt="NPM Version" height="28"></a>
    <span>&nbsp;</span>
    <a href="https://pypi.org/project/julep"><img src="https://img.shields.io/pypi/v/julep?style=social&amp;logo=python&amp;label=PyPI&amp;link=https%3A%2F%2Fpypi.org%2Fproject%2Fjulep" alt="PyPI - Version" height="28"></a>
    <span>&nbsp;</span>
    <a href="https://hub.docker.com/u/julepai"><img src="https://img.shields.io/docker/v/julepai/agents-api?sort=semver&amp;style=social&amp;logo=docker&amp;link=https%3A%2F%2Fhub.docker.com%2Fu%2Fjulepai" alt="Docker Image Version" height="28"></a>
    <span>&nbsp;</span>
    <a href="https://choosealicense.com/licenses/apache/"><img src="https://img.shields.io/github/license/julep-ai/julep" alt="GitHub License" height="28"></a>
  </p>
  
  <h3>
    <a href="https://discord.com/invite/JTSBGRZrzj" rel="dofollow">Discord</a>
    ·
    <a href="https://x.com/julep_ai" rel="dofollow">𝕏</a>
    ·
    <a href="https://www.linkedin.com/company/julep-ai" rel="dofollow">LinkedIn</a>
  </h3>
</div>

## Task: Travel Itinerary Assistant

### Overview

The Travel Itinerary Assistant helps users plan a travel itinerary that takes into account **current weather conditions** and **local tourist attractions**. By integrating data from Wikipedia for tourist attractions and using a weather API for real-time weather updates, the system provides a comprehensive travel plan tailored to each location. The generated itinerary suggests appropriate activities based on the weather, enhancing the overall travel experience.

### Task Flow

1. **User Input**
   - User provides a list of desired travel locations.
   - Each location is processed individually to gather the required data.

2. **Weather Data Retrieval**
   - Fetch current weather data for each location using a weather API.
   - Extract relevant weather details, such as temperature, weather condition, and recommendations.

3. **Tourist Attractions Lookup**
   - Use Wikipedia to search for the top tourist attractions for each location.
   - The query format used is: `"<location> tourist attractions"`.
   - Retrieve and compile a list of popular tourist spots and landmarks.

4. **Data Evaluation and Integration**
   - Combine weather data and tourist attractions into a unified list for each location.
   - Format the data into a structured tuple: `(location, weather, attractions)`.

5. **Itinerary Generation**
   - Create a detailed travel itinerary based on:
     - Current weather conditions (e.g., sunny, rainy, cloudy).
     - Top tourist attractions for each location.
     - Suggested activities categorized as indoor or outdoor based on weather.

### Key Features

- **Multi-location Travel Planning**: Handles multiple destinations simultaneously, offering a consolidated travel plan.
- **Real-time Weather Data**: Leverages weather APIs to provide up-to-date weather conditions.
- **Tourist Attraction Discovery**: Integrates Wikipedia to find and recommend popular attractions.
- **Intelligent Itinerary Suggestions**: Suggests indoor or outdoor activities based on the weather.
- **Comprehensive Itinerary Output**: Combines weather and tourist data into a user-friendly travel plan.

### Output

- A detailed travel itinerary for each location
- Curated, up-to-date information gathered from weather searches and Wikipedia

```plaintext

 +----------------+     +--------------------------+     +--------------------------+     +------------------------------+     +------------------+
 |   User Input   |     | Weather Data Retrieval   |     | Tourist Attractions      |     | Data Evaluation & Integration|     | Itinerary        |
 |    (List of    | --> |  (Weather API)           | --> | Lookup (Wikipedia)       | --> |    (Combine Weather &        | --> | Generation       |
 |    Locations)  |     |                          |     |                          |     |   Attractions Data)          |     | (Generate Plan)  |
 +----------------+     +--------------------------+     +--------------------------+     +------------------------------+     +------------------+
        |                              |                             |                                   |                              |
        |                              |                             |                                   |                              |
        v                              v                             v                                   v                              v
Location 1, Location 2, ...   Fetch weather for each       Search Wikipedia for           Combine weather data and         Create itinerary with
Each location processed       location individually,       "<location> tourist            tourist attractions into          suggested activities
individually for              extracting temp.,            attractions", retrieve         a structured tuple:              based on weather and
weather data.                 conditions, &                top spots.                     (location, weather,              attractions.
                               recommendations.                                             attractions).
```




## Implementation

To recreate the notebook and see the code implementation for this task, you can access the Google Colab notebook using the link below:

<a target="_blank" href="https://colab.research.google.com/github/julep-ai/julep/blob/dev/cookbooks/03-trip-planning-assistant.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Additional Information

For more details about the task or if you have any questions, please don't hesitate to contact the author:

**Author:** Julep AI  
**Contact:** [hey@julep.ai](mailto:hey@julep.ai) or  <a href="https://discord.com/invite/JTSBGRZrzj" rel="dofollow">Discord</a>

Installing the Julep Client

In [24]:
!pip install julep -U --quiet

#### NOTE:

- UUIDs are generated for both the agent and task to uniquely identify them within the system.
- Once created, these UUIDs should remain unchanged for simplicity.
- Altering a UUID will result in the system treating it as a new agent or task.
- If a UUID is changed, the original agent or task will continue to exist in the system alongside the new one.

In [25]:
# Global UUID is generated for agent and task
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4() 

### Creating Julep Client with the API Key

Get your API key from [here](https://dashboard.julep.ai/)

In [26]:
from julep import Client
import os

JULEP_API_KEY = os.environ["JULEP_API_KEY"]

# Create a Julep client
client = Client(api_key=JULEP_API_KEY, environment="production")

### Creating an "agent"

Agent is the object to which LLM settings, like model, temperature along with tools are scoped to.

To learn more about the agent, please refer to the Agent section in [Julep Concepts](https://docs.julep.ai/docs/concepts/agents).

In [27]:
# Defining the agent
name = "Jacob"
about = "A travel assistant that helps plan the perfect trip."

# Create the agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name=name,
    about=about,
    model="gpt-4o",
)

### Defining a Task

Tasks in Julep are Github-Actions-style workflows that define long-running, multi-step actions.

You can use them to conduct complex actions by defining them step-by-step.

To learn more about tasks, please refer to the `Tasks` section in [Julep Concepts](https://docs.julep.ai/docs/concepts/tasks).

In [45]:
import yaml

openweathermap_api_key = "YOUR_API_KEY"
brave_api_key = "YOUR_API_KEY"

# Defining the task
task_def = yaml.safe_load(f"""
# yaml-language-server: $schema=https://raw.githubusercontent.com/julep-ai/julep/refs/heads/dev/schemas/create_task_request.json
name: Julep Tourist Plan With Weather And Attractions
description: A task that plans a trip with weather and attractions.

########################################################
####################### INPUT SCHEMA ##################
########################################################                       
input_schema:
  type: object
  properties:
    locations:
      type: array
      items:
        type: string
      description: The locations to search for.

########################################################
####################### TOOLS ##########################
########################################################

# Define the tools that the task will use in this workflow
tools:
- name: wikipedia
  type: integration
  integration:
    provider: wikipedia

- name: weather
  type: integration
  integration:
    provider: weather
    setup:
      openweathermap_api_key: {openweathermap_api_key}

- name: internet_search
  type: integration
  integration:
    provider: brave
    setup:
      brave_api_key: {brave_api_key}

########################################################
####################### MAIN WORKFLOW ##########################
########################################################

main:
# Step 0: Fetch weather data for each location
- over: $ steps[0].input.locations
  map:
    tool: weather
    arguments:
      location: $ _

# Step 1: Search Wikipedia for tourist attractions for each location
- over: $ steps[0].input.locations
  map:
    tool: internet_search
    arguments:
      query: $ 'tourist attractions in ' + _

# Step 2: Zip locations, weather, and attractions into a list of tuples [(location, weather, attractions)]
- evaluate:
    zipped: |-
      $ list(
        zip(
          steps[0].input.locations,
          [output['result'] for output in steps[0].output],
          steps[1].output
        )
      )

# Step 3: Create an itinerary for each location
- over: $ _['zipped']
  parallelism: 3
  # Inside the map step, each `_` represents the current element in the list
  # which is a tuple of (location, weather, attractions)
  map:
    prompt:
    - role: system
      content: >-
        $ f'''You are {{agent.name}}. Your task is to create a detailed itinerary
        for visiting tourist attractions in some locations.
        The user will give you the following information for each location:

        - The location
        - The current weather condition
        - The top tourist attractions'''
    - role: user
      content: >-
        $ f'''Location: "{{_[0]}}"
        Weather: "{{_[1]}}"
        Attractions: "{{_[2]}}"'''
    unwrap: true

# Step 4: Create a final plan by joining the activities for each location
- evaluate:
    final_plan: |-
      $ '\\n---------------\\n'.join(activity for activity in _)
""")

<span style="color:olive;">Notes:</span>
- The `unwrap: True` in the prompt step is used to unwrap the output of the prompt step (to unwrap the `choices[0].message.content` from the output of the model).
- The `$` sign is used to differentiate between a Python expression and a string.
- The `_` refers to the output of the previous step.
- The `steps[index].input` refers to the input of the step at `index`.
- The `steps[index].output` refers to the output of the step at `index`.

In [46]:
# creating the task object
task = client.tasks.create_or_update(
    task_id=TASK_UUID,
    agent_id=AGENT_UUID,
    **task_def
)

### Creating an Execution

An execution is a single run of a task. It is a way to run a task with a specific set of inputs.

To learn more about executions, please refer to the `Executions` section in [Julep Concepts](https://docs.julep.ai/docs/concepts/execution).

In [47]:
execution = client.executions.create(
    task_id=task.id,
    input={
         "locations": ["New York", "London", "Paris", "Tokyo", "Sydney"]
    }
)

print("Started an execution. Execution ID:", execution.id)

Started an execution. Execution ID: 067ffa47-3b83-79b7-8000-eaaa1a013f0e


## Checking execution details and output

There are multiple ways to get the execution details and the output:

1. **Get Execution Details**: This method retrieves the details of the execution, including the output of the last transition that took place.

2. **List Transitions**: This method lists all the task steps that have been executed up to this point in time, so the output of a successful execution will be the output of the last transition (first in the transition list as it is in reverse chronological order), which should have a type of `finish`.


<span style="color:olive;">Note: You need to wait for a few seconds for the execution to complete before you can get the final output, so feel free to run the following cells multiple times until you get the final output.</span>


In [48]:
import time

execution = client.executions.get(execution.id)

while execution.status != "succeeded":
    time.sleep(5)
    execution = client.executions.get(execution.id)
    print("Execution status: ", execution.status)
    print("-"*50)

execution = client.executions.get(execution.id)

if 'final_plan' in execution.output:
    print(execution.output['final_plan'])
else:
    print(execution.output)


Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  succeeded
--------------------------------------------------
Here's a detailed itinerary for exploring New York, tailored to the current weather and top attractions:

### Day 1: Arrival and Exploration

**Morning:**
- **Central Park:**
  - Begin your day with a brisk walk through Central Park. It's a perfect way to enjoy the morning with temperatures arou

In [49]:
# Lists all the task steps that have been executed up to this point in time
transitions = client.executions.transitions.list(execution_id=execution.id).items

# Transitions are retrieved in reverse chronological order
for transition in reversed(transitions):
    print("Transition type: ", transition.type)
    print("Transition output: ", transition.output)
    print("-"*50)

Transition type:  init
Transition output:  {'locations': ['New York', 'London', 'Paris', 'Tokyo', 'Sydney']}
--------------------------------------------------
Transition type:  init_branch
Transition output:  Sydney
--------------------------------------------------
Transition type:  init_branch
Transition output:  London
--------------------------------------------------
Transition type:  init_branch
Transition output:  Tokyo
--------------------------------------------------
Transition type:  init_branch
Transition output:  Paris
--------------------------------------------------
Transition type:  init_branch
Transition output:  New York
--------------------------------------------------
Transition type:  finish_branch
Transition output:  {'result': 'In London, the current weather is as follows:\nDetailed status: scattered clouds\nWind speed: 8.05 m/s, direction: 222°\nHumidity: 45%\nTemperature: \n  - Current: 14.87°C\n  - High: 15.83°C\n  - Low: 13.86°C\n  - Feels like: 13.59°C\nR